In [1]:
import numpy as np
import pandas as pd
from bigbio.dataloader import BigBioConfigHelpers

In [3]:
st21pv_sources = ['CPT','FMA','GO','HGNC','ICD10','ICD10CM','ICD9CM','MDR','MSH','MTH','NCBI','NCI','NDDF','NDFRT','OMIM','RXNORM','SNOMEDCT_US']
col_names = ['cui','lang','term_status','lui','stt','sui','ispref','aui','saui','scui','sdui','sab','tti','code','alias','srl','suppress','cvf','null_col']
df = pd.read_csv('/efs/davidkartchner/2022AA/META/MRCONSO.RRF', delimiter='|', names=col_names, usecols=['cui','lang','sab','alias'])

cui       0
lang      0
sab       0
alias    24
dtype: int64

In [9]:
eng_filtered = df.query('lang == "ENG"').dropna()
eng_filtered['alias'] = eng_filtered.alias.map(lambda x: x.lower())

In [10]:
filtered = eng_filtered[eng_filtered['sab'].isin(st21pv_sources)].drop_duplicates()
output = filtered[['cui','alias']].drop_duplicates()


TypeError: 'tuple' object is not callable

In [14]:
output = filtered[['cui','alias']].drop_duplicates()
print(output.shape)
filtered.head()


(6737604, 2)


,cui,lang,sab,alias
0,C0000005,ENG,MSH,(131)i-macroaggregated albumin
1,C0000005,ENG,MSH,(131)i-maa
10,C0000039,ENG,RXNORM,"1,2-dipalmitoylphosphatidylcholine"
11,C0000039,ENG,MTH,"1,2-dipalmitoylphosphatidylcholine"
12,C0000039,ENG,MSH,"1,2-dipalmitoylphosphatidylcholine"


In [15]:
output.head()

,cui,alias
0,C0000005,(131)i-macroaggregated albumin
1,C0000005,(131)i-maa
10,C0000039,"1,2-dipalmitoylphosphatidylcholine"
13,C0000039,"1,2 dipalmitoylphosphatidylcholine"
14,C0000039,"1,2-dihexadecyl-sn-glycerophosphocholine"


In [16]:
np.savetxt('/efs/davidkartchner/sapbert/data/medmentions/umls_dict_uncased.txt', output, delimiter='||', fmt='%s', comments='', encoding=None)

## Process MedMentions dataset to SAPBERT format

In [3]:
conhelps = BigBioConfigHelpers()
medmentions = conhelps.for_config_name('medmentions_st21pv_bigbio_kb').load_dataset()


Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset med_mentions_dataset downloaded and prepared to /home/davidkartchner/.cache/huggingface/datasets/med_mentions_dataset/medmentions_st21pv_bigbio_kb/1.0.0/52b27a04ca8f065ee2eaa96a56bea0f4aac8f3b43c3a6516e0b3c985b2657448. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '~/entity_linking_review/sapbert/data/medmentions/processed_test/25847295.concept'

In [8]:

for split in ['train','test','validation']:
    for abs in medmentions[split]:
        pmid = abs['document_id']
        all_ents = []
        for e in abs['entities']:
            text = e['text'][0]
            offsets = '|'.join([str(i) for i in e['offsets'][0]])
            cui = e['normalized'][0]['db_id'].split(':')[-1]
            tui = e['type']
            all_ents.append('||'.join([pmid, offsets, tui, text, cui]))
        with open(f'/home/davidkartchner/entity_linking_review/sapbert/data/medmentions/processed_{split}/{pmid}.concept', 'w') as f:
            f.write('\n'.join(all_ents))
    


## Evaluate SapBERT